### <span style="color:#0ab49a">Занятие №5:</span> <span style="color:#BA77D8">Немного чистой архитектуры</span> 

![](img/banner.png)

> Каждая минута, потраченная на организацию своей деятельности, экономит вам целый час
>> Бенджамин Франклин

### <span style="color:#55628D">1. Выводы из предыдущего занятия</span>

1. Большой код можно и нужно разбивать на модули; модули включают друг друга с помощью *import*
2. Множество модулей можно импортировать скопом с помощью *\_\_init\_\_.py*
3. При взаимном импорте модулей со стороны Python идёт "тонкая настройка", которую можно пониимать или избегать

**Это ответы на вопросы "Как делать можно". А как делать надо/не надо?**

#### <span style="color:#1DA398">Разделение проекта на уровни</span>

![](img/pure_architecure.jpeg)
![](img/pure_architecture.png)

#### <span style="color:#1DA398">Пример: комплекс ориентации и навигации (КОН)</span>

![](img/КОН.jpg)

### <span style="color:#55628D">2. Принципы SOLID</span>

#### <span style="color:#1DA398">2.1 Принцип единственной ответственности</span>
##### <span style="color:#737eb5">У каждого класса только 1 ответственность; у него нет других обязанностей</span>
**Пример:** калькулятор, который достаёт числа из файла и записывает в файл

In [ ]:
# Плохой пример
class CountManagment_And_FileManagment:
    def __init__(self, name_save, name_load):
        self.a = None
        self.b = None
        self.c = None
        self.name_save = name_save
        self.name_load = name_load

    def load(self):
        with open(self.name_load, "r") as f:
            ab = f.read().split()
        self.a = int(ab[0])
        self.b = int(ab[1])

    def save(self):
        with open(self.name_save, "w") as f:
            f.write(self.c)

    def sum(self):
        self.c = self.a + self.b

In [ ]:
# Хороший пример
class CountManagment:
    def __init__(self):
        self.a = None
        self.b = None
        self.c = None

    def sum(self):
        self.c = self.a + self.b


class FileManagment:
    def __init__(self, name_save, name_load):
        self.name_save = name_save
        self.name_load = name_load

    def load(self, counter: CountManagment):
        with open(self.name_load, "r") as f:
            ab = f.read().split()
        counter.a = int(ab[0])
        counter.b = int(ab[1])

    def save(self, counter: CountManagment):
        with open(self.name_save, "w") as f:
            f.write(counter.c)

#### <span style="color:#1DA398">2.2 Принцип открытости/закрытости</span>
##### <span style="color:#737eb5">Сущности (классы, модули, функции) октрыты для расширения, но закрыты для изменений</span>
**Пример:** рассчёт физических величин из констант

In [ ]:
# Плохой пример
def get_circle_len(r):
    pi = 3.1415
    return 2 * pi * r

In [ ]:
# Хороший пример
import math
pi_1 = 3.1415
pi_2 = math.pi

def get_circle_len(r, pi):
    return 2 * pi * r

#### <span style="color:#1DA398">2.3 Принцип подстановки Лисков</span>
##### <span style="color:#737eb5">Объекты в программе должны быть заменяемы экземплярами их подтипов без ущерба корректности работы программы</span>
или
##### <span style="color:#737eb5">Функции, которые используют базовый тип, должны иметь возможность использовать подтипы базового типа, не зная об этом.</span>
**Пример:** подкласс машины, на которой дополнительный двигатель

In [4]:
# Хороший пример
class Car:
    dt = 0.1
    
    def __init__(self, horsepower):
        self.x = 0.
        self.v = 0.
        self.a = horsepower

    def run(self, times):
        for _ in range(times):
            self.v += self.a * self.dt
            self.x += self.v * self.dt

class HybridCar(Car):
    def __init__(self, horsepower, extra_horsepower):
        super().__init__(horsepower)
        self.extra_a = extra_horsepower

    def run(self, times):
        for _ in range(times):
            self.v += (self.a + self.extra_a) * self.dt
            self.x += self.v * self.dt

In [5]:
c = Car(10)
c.run(100)
print(c.x)

505.0


In [6]:
c = HybridCar(10, 1)
c.run(100)
print(c.x)

555.4999999999998


#### <span style="color:#1DA398">2.4 Принцип разделения интерфейса</span>
##### <span style="color:#737eb5">Ни один клиент не должен зависеть от методов, которые он не использует</span>
**Пример:** "слишком раздутый" интефрейс у общего класса транспортных средств

In [11]:
# Плохой пример
class Vehicle:
    dt = 0.1
    
    def __init__(self, tag, horsepower):
        self.x = 0.
        self.v = 0.
        self.tag = tag
        self.a = horsepower

    def run(self, times):
        for _ in range(times):
            self.v += self.a * self.dt
            self.x += self.v * self.dt

    def sign(self):
        print(self.tag + ": БИИИИИИИИИИП!!!")

class Plane(Vehicle):
    pass

p = Plane('Ту-160', 100)
p.sign()

Ту-160: БИИИИИИИИИИП!!!


In [19]:
# Хороший пример
class Vehicle:
    dt = 0.1
    
    def __init__(self, tag, horsepower):
        self.x = 0.
        self.v = 0.
        self.tag = tag
        self.a = horsepower

    def run(self, times):
        for _ in range(times):
            self.v += self.a * self.dt
            self.x += self.v * self.dt

class SignDevice:
    def __init__(self, tag):
        self.tag = tag
        
    def sign(self):
        print(self.tag + ": БИИИИИИИИИИП!!!")

class Car(Vehicle, SignDevice):
    pass

class Plane(Vehicle):
    pass

c = Car(tag='VW Polo', horsepower=10)
c.sign()

VW Polo: БИИИИИИИИИИП!!!


In [ ]:
p = Plane('Ту-160', 100)
p.sign()

#### <span style="color:#1DA398">2.5 Принцип инверсии зависимостей</span>
##### <span style="color:#737eb5">Абстракции не должны зависеть от деталей. Детали должны зависеть от абстракций.</span>
Неявная согласованность с принципом подстановки Лисков

**Пример:** тестирование калькулятора

In [7]:
# Плохой пример
class Calculator:
    def set_a_b(self, a, b):
        self.a = a
        self.b = b

    def get_sum(self):
        return self.a + self.b + 1

def test_calc():
    a = 1
    b = 2

    c = Calculator()
    c.set_a_b(a=a, b=b)
    if c.get_sum() == a + b:
        print(f"Калькулятор работает верно!")
    else:
        print(f"Калькулятор работает неверно! ({c.get_sum()} != {a + b})")

test_calc()

Калькулятор работает неверно! (4 != 3)


In [6]:
# Хороший пример
class Calculator:
    def set_a_b(self, a, b):
        self.a = a
        self.b = b

    def get_sum(self, a=None, b=None):
        if a is None and b is None:
            a, b = self.a, self.b
        return a + b + 1
        

def test_calc():
    a = 1
    b = 2

    if Calculator().get_sum(a, b) == a + b:
        print(f"Калькулятор работает верно!")
    else:
        print(f"Калькулятор работает неверно! ({Calculator().get_sum(a, b)} != {a + b})")

test_calc()

Калькулятор работает неверно! (4 != 3)


### <span style="color:#48B026"> Пример контрольной #3 (2021 год)</span>
Решение задач на семинаре

##### Строка

##### Фонари

##### Толстые коты

##### Шарики

##### События в сигнале

##### Покемоны

##### Опечатки

##### Дискорд

### <span style="color:#48B026"> Пример контрольной #4 (2021 год)</span>

### <span style="color:#0ab49a">Примечание №1.</span> <span style="color:#BA77D8">Перед контрольной</span> 

In [12]:
# Вывод текста определённой длинны
a = 1.23
n = 4
a_1, a_2 = str(a).split(".")
print(type(a_1), a_1, type(a_2), a_2)

<class 'str'> 1 <class 'str'> 23


In [15]:
# Способ вывести полностью вручную
print(a_1 + '.' + a_2 + '0'*(n - len(a_2)))

# Способ с использованием rjust(), ljust()
print(str(a).ljust(n + 1 + len(a_1), '0'))

# Новый способ
print(("{:." + str(n) + "f}").format(a))

# Новейший способ
print(f"{a:.4f}")
print(f"{a:.{n}f}")

1.2300
1.2300
1.2300
1.2300
1.2300
